# Lumerical FDTD Sparameters

gdsfactory provides you with a Lumerical FDTD interface to calculate Sparameters automatically  (without you having to click around the Lumerical GUI)

The function `gdsfactory.simulation.write_sparameters_lumerical` brings up a GUI, runs simulation and then writes the Sparameters both in .CSV and .DAT (Lumerical interconnect / simphony) file formats, as well as the simulation settings in YAML format.

In the CSV format each Sparameter will have 2 columns, `S12m` where `m` stands for magnitude and `S12a` where `a` stands for angle in radians.

For the simulation to wor well, your components need to have ports, that will be extended automatically to go over the PML.

![](https://i.imgur.com/dHAzZRw.png)


The script calls internally the lumerical python API `lumapi` so you will need to make sure that you can run this from python.

```python
import lumapi

session = luampi.FDTD()
```

In linux that may require you to export the PYTHONPATH variable in your shell environment.

You can add one line into your `.bashrc` in your Linux machine. This line will depend also on your Lumerical version. For example for Lumerical 2019b




```bash
[ -d "/opt/lumerical/2019b" ] && export PATH=$PATH:/opt/lumerical/2019b/bin && export PYTHONPATH=/opt/lumerical/2019b/api/python
```


And for 2021v212

```bash
[ -d "/opt/lumerical/v212" ] && export PATH=$PATH:/opt/lumerical/v212/api/python/bin && export PYTHONPATH=/opt/lumerical/v212/api/python
```


Finally, You can chain the Sparameters to calculate solve of larger circuits using a circuit solver such as:

- Lumerical interconnect
- [simphony (open source)](https://simphonyphotonics.readthedocs.io/en/latest/)
- [sax (open source)](https://sax.readthedocs.io/en/latest/index.html)



In [ ]:
import gdsfactory as gf
import gdsfactory.simulation as sim

gf.tech.SIMULATION_SETTINGS

In [ ]:
sim.write_sparameters_lumerical?

In [ ]:
# NBVAL_SKIP
import gdsfactory as gf
import gdsfactory.simulation as sim
import lumapi

s = lumapi.FDTD()

In [ ]:
gf.components.factory.keys()

In [ ]:
components = [
    "bend_euler",
    "bend_s",
    "coupler",
    "coupler_ring",
    "crossing",
    "mmi1x2",
    "mmi2x2",
    "taper",
    "straight",
]
need_review = []

for component_name in components:
    component = gf.components.factory[component_name]()
    sim.write_sparameters_lumerical(component, run=False, session=s)
    response = input(f"does the simulation for {component_name} look good? (y/n)")
    if response.upper()[0] == "N":
        need_review.append(componnent_name)

In [ ]:
components = ["crossing", "mmi1x2", "mmi2x2", "taper", "straight"]
need_review = []

for component_name in components:
    component = gf.components.factory[component_name]()
    sim.write_sparameters_lumerical(component, run=False, session=s)
    response = input("does the simulation look good? (y/n)")
    if response.upper()[0] == "N":
        need_review.append(component_name)

In [ ]:
sim.plot.plot_sparameters(gf.components.mmi1x2(), keys=["S23m", "S13m"], logscale=True)

In [ ]:
sim.write_sparameters_lumerical(gf.components.mmi1x2(), layer_to_thickness={(1, 0): 210e-3})

In [ ]:
layer_to_thickness = gf.tech.LAYER_STACK.get_layer_to_thickness()
component = gf.components.bend_circular(radius=3)
component = component.copy()
component.remove_layers(component.layers - set(layer_to_thickness.keys()))
component._bb_valid = False

c = gf.components.extension.extend_ports(component=component, length=1.0)
c

gdsfactory can also compute the Sparameters of a component that have not been simulated before.

In [ ]:
# NBVAL_SKIP

cs = [
    gf.components.coupler_ring(gap=gap, radius=radius)
    for gap in [0.15, 0.2, 0.3]
    for radius in [5, 10]
]

for c in cs:
    c.show()
    print(c)
    sim.write_sparameters_lumerical(c)

To debug a simulation you can create a Lumerical session outside the simulator, pass it to the simulator, and use `run=False` flag

In [ ]:
# NBVAL_SKIP
import lumapi
import gdsfactory as gf

s = lumapi.FDTD()
c = gf.components.straight()
sim.write_sparameters_lumerical(c, run=False, session=s)

By default gdsfactory uses the generic LayerStack for 0.22um height silicon layer.

You can also define your a different LayerStack

In [ ]:
import gdsfactory as gf

def get_layer_stack():
    return gf.tech.LayerStack(wg=gf.tech.LayerLevel(layer=(2, 0), thickness=400e-3, zmin=0.0, material="sin")


ls = get_layer_stack()

In [ ]:
# NBVAL_SKIP
import gdsfactory as gf
import gdsfactory.simulation as sim

c = gf.components.straight()
s = sim.write_sparameters_lumerical(c, run=False)

In [ ]:
s.select('FDTD')

In [ ]:
s.set('simulation time', 10e-9)

In [ ]:
s.set('simulation temperature', 350)

In [ ]:
# speed*time = distance
3e8/4.2*1e-12*1e6